In [9]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [10]:
# Load and preprocess the MNIST data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape((x_train.shape[0], 28, 28, 1))
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1))

In [11]:
# Resize images to 32x32 to match the input shape of ResNet50
x_train = tf.image.resize(x_train, [32, 32])
x_test = tf.image.resize(x_test, [32, 32])

In [12]:
# One-hot encode the labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [13]:
# Load the ResNet50 model with pre-trained ImageNet weights, excluding the top layers
base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))



In [14]:
# Add custom layers on top of ResNet50
model = models.Sequential()
model.add(layers.Conv2D(3, (1, 1), padding='same', input_shape=(32, 32, 1))) # Convert grayscale to RGB
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))


In [15]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [16]:
# Callbacks
checkpoint = ModelCheckpoint('best_mnist_resnet50_model.h5', monitor='val_accuracy', save_best_only=True, mode='max')
early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min')


In [17]:
# Train the model with callbacks
history = model.fit(x_train, y_train, epochs=20, batch_size=64, validation_data=(x_test, y_test), callbacks=[checkpoint, early_stopping])



Epoch 1/20
938/938 [==============================] - ETA: 0s - loss: 0.1538 - accuracy: 0.9614

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


938/938 [==============================] - 91s 63ms/step - loss: 0.1538 - accuracy: 0.9614 - val_loss: 52.6626 - val_accuracy: 0.6138
Epoch 2/20
938/938 [==============================] - 53s 57ms/step - loss: 0.1048 - accuracy: 0.9747 - val_loss: 0.1381 - val_accuracy: 0.9618
Epoch 3/20
938/938 [==============================] - 59s 63ms/step - loss: 0.0541 - accuracy: 0.9867 - val_loss: 0.0532 - val_accuracy: 0.9875
Epoch 4/20
938/938 [==============================] - 59s 63ms/step - loss: 0.0587 - accuracy: 0.9857 - val_loss: 0.0440 - val_accuracy: 0.9888
Epoch 5/20
938/938 [==============================] - 57s 61ms/step - loss: 0.0414 - accuracy: 0.9897 - val_loss: 0.0244 - val_accuracy: 0.9929
Epoch 6/20
938/938 [==============================] - 51s 54ms/step - loss: 0.0387 - accuracy: 0.9903 - val_loss: 4.4484 - val_accuracy: 0.8796
Epoch 7/20
938/938 [==============================] - 52s 56ms/step - loss: 0.1114 - accuracy: 0.9734 - val_loss: 1.8615 - val_accuracy: 0.8514
Ep

In [18]:
# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test accuracy: {accuracy*100:.2f}%")

313/313 [==============================] - 4s 12ms/step - loss: 0.0316 - accuracy: 0.9913
Test accuracy: 99.13%


In [19]:
# Save the trained model
model.save('mnist_resnet50_model.h5')

